In [1]:
import pandas as pd
import numpy as np

import data_cleaning as dc

In [20]:
df = dc.ems_data()
df_2020 = dc.limit_dates(df, '01/01/2020', '12/31/2020')
df_2019 = dc.limit_dates(df, '01/01/2019', '12/31/2019')
df_2015_2019 = dc.limit_dates(df, '01/01/2015', '12/31/2019')

C:\Users\Jeffrey\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (7,11,17,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Jeffrey\Desktop\Homework\Resource-Allocation-of-Emergency-Medical-Services\data_cleaning.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INCIDENT_DATE'] = df['INCIDENT_CLOSE_DATETIME'].dt.date


Time Points
- t1: Incident created
- t2: Incident assigned
- t3: Ambulance arrival
- t4: Hospital arrival
- t5: Incident closed

Time Variables
- t1 to t2: Dispatch response time
- t2 to t3: Travel time
- t1 to t3: Incident response time
- t1 to t5: Total time

Dispatches
- 9 dispatch locations in Manhattan
    - {M1, M2, ... M9}
- 4 without location
    - {CW, T1, X2, X4}

Codes
- 82: Transporting Patient
- 93: Refused Medical Aid


In [3]:
df_2020.columns

Index(['CAD_INCIDENT_ID', 'INCIDENT_DATETIME', 'INITIAL_CALL_TYPE',
       'INITIAL_SEVERITY_LEVEL_CODE', 'FINAL_CALL_TYPE',
       'FINAL_SEVERITY_LEVEL_CODE', 'FIRST_ASSIGNMENT_DATETIME',
       'VALID_DISPATCH_RSPNS_TIME_INDC', 'DISPATCH_RESPONSE_SECONDS_QY',
       'FIRST_ACTIVATION_DATETIME', 'FIRST_ON_SCENE_DATETIME',
       'VALID_INCIDENT_RSPNS_TIME_INDC', 'INCIDENT_RESPONSE_SECONDS_QY',
       'INCIDENT_TRAVEL_TM_SECONDS_QY', 'FIRST_TO_HOSP_DATETIME',
       'FIRST_HOSP_ARRIVAL_DATETIME', 'INCIDENT_CLOSE_DATETIME',
       'HELD_INDICATOR', 'INCIDENT_DISPOSITION_CODE', 'BOROUGH',
       'INCIDENT_DISPATCH_AREA', 'ZIPCODE', 'POLICEPRECINCT',
       'CITYCOUNCILDISTRICT', 'COMMUNITYDISTRICT', 'COMMUNITYSCHOOLDISTRICT',
       'CONGRESSIONALDISTRICT', 'REOPEN_INDICATOR', 'SPECIAL_EVENT_INDICATOR',
       'STANDBY_INDICATOR', 'TRANSFER_INDICATOR', 'INCIDENT_DATE'],
      dtype='object')

In [4]:
df_2020['INCIDENT_DISPATCH_AREA'].unique()

array(['M1', 'M9', 'M7', 'M5', 'M2', 'M3', 'X1', 'CW', 'M8', 'M4', 'M6',
       'K2', 'B5', 'B2', 'Q6', 'K3', 'K4', 'Q1', 'K6', 'K7', 'Q5', 'K5',
       'Q3', 'K1', 'B3', 'S1', 'B1', 'S2', 'Q7', 'Q4', 'Q2'], dtype=object)

In [21]:
l_2020, l_zip_2020, l_2019, l_zip_2019 = [], [], [], []

print('Median 2020: '+str(round(df_2020['INCIDENT_RESPONSE_SECONDS_QY'].dropna().count(), 2)))
print('  Median by ZIP 2020')
for i in df_2020['COMMUNITYDISTRICT'].dropna().unique():
    print('    '+str(i)[:-2]+': '+ str(df_2020.loc[df_2020['COMMUNITYDISTRICT']==i]['INCIDENT_RESPONSE_SECONDS_QY'].dropna().count()))
    l_2020.append(df_2020.loc[df_2020['COMMUNITYDISTRICT']==i]['INCIDENT_RESPONSE_SECONDS_QY'].dropna().count())
    l_zip_2020.append(str(i)[:-2])

print('\nMedian 2019: '+str(round(df_2019['INCIDENT_RESPONSE_SECONDS_QY'].dropna().count())))
print('  Median by ZIP 2019')
for i in df_2019['COMMUNITYDISTRICT'].dropna().unique():
    print('    '+str(i)[:-2]+': '+ str(df_2019.loc[df_2019['COMMUNITYDISTRICT']==i]['INCIDENT_RESPONSE_SECONDS_QY'].dropna().count()))
    l_2019.append(df_2019.loc[df_2019['COMMUNITYDISTRICT']==i]['INCIDENT_RESPONSE_SECONDS_QY'].dropna().count())
    l_zip_2019.append(str(i)[:-2])

Median 2020: 142423
  Median by ZIP 2020
    101: 6609
    111: 18838
    108: 8560
    105: 13897
    103: 14661
    104: 13610
    112: 15277
    102: 5861
    110: 14287
    106: 9825
    107: 11080
    109: 8911
    164: 110
    208: 2

Median 2019: 149944
  Median by ZIP 2019
    103: 15156
    111: 18545
    110: 13730
    108: 9489
    107: 11654
    105: 18063
    112: 12875
    101: 7777
    104: 15679
    106: 10643
    109: 7756
    102: 7272
    164: 158
    401: 1
    208: 4


In [33]:
z_2020, z_2019 = pd.DataFrame({'COMMUNITYDISTRICT': l_zip_2020, '2020': l_2020}), pd.DataFrame({'COMMUNITYDISTRICT': l_zip_2019, '2019': l_2019})
zip_medians = z_2020.merge(z_2019, on='COMMUNITYDISTRICT')
#zip_medians['COMMUNITYDISTRICT'] = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5]
zip_medians.to_csv('cds.csv', index=False)
zip_medians

,COMMUNITYDISTRICT,2020,2019
0,101,6609,7777
1,111,18838,18545
2,108,8560,9489
3,105,13897,18063
4,103,14661,15156
5,104,13610,15679
6,112,15277,12875
7,102,5861,7272
8,110,14287,13730
9,106,9825,10643


In [11]:
subset = df_2020[['INCIDENT_DISPATCH_AREA', 'INCIDENT_RESPONSE_SECONDS_QY', 'COMMUNITYDISTRICT']].dropna()
subset['TEST'] = [x[:-1] for x in subset['INCIDENT_DISPATCH_AREA']]
subset = subset.loc[subset['TEST'] == 'M']
communities = [subset['COMMUNITYDISTRICT'].unique(), [1, 3, 3, 2, 1, 2, 4, 1, 4, 2, 3, 4, 5]]
communities = pd.DataFrame({'COMMUNITYDISTRICT': communities[0], 'COMBO_COMMUNITY': communities[1]})
communities = communities.loc[communities['COMBO_COMMUNITY']!=5]

combo = subset.merge(communities, on='COMMUNITYDISTRICT')[['INCIDENT_DISPATCH_AREA', 'INCIDENT_RESPONSE_SECONDS_QY', 'COMBO_COMMUNITY']]
combo.groupby(['INCIDENT_DISPATCH_AREA', 'COMBO_COMMUNITY']).mean().reset_index().to_csv('dispatch_community_distance.csv', index=False)

In [12]:
# Calculate the average and max values for each dispatch location in 2020
subset_2020 = df_2020[['CAD_INCIDENT_ID', 'INCIDENT_DATE', 'INCIDENT_DISPATCH_AREA']]
subset_grouped = subset_2020.groupby(['INCIDENT_DATE', 'INCIDENT_DISPATCH_AREA']).count().reset_index()
subset_median_daily = subset_grouped.groupby(['INCIDENT_DISPATCH_AREA']).median()['CAD_INCIDENT_ID']
subset_max_daily = subset_grouped.groupby(['INCIDENT_DISPATCH_AREA']).max()['CAD_INCIDENT_ID']

subset_2015_2019 = df_2015_2019[['CAD_INCIDENT_ID', 'INCIDENT_DATE', 'INCIDENT_DISPATCH_AREA']]
subset_grouped_2015_2019 = subset_2015_2019.groupby(['INCIDENT_DATE', 'INCIDENT_DISPATCH_AREA']).count().reset_index()
subset_median_daily_2015_2019 = subset_grouped_2015_2019.groupby(['INCIDENT_DISPATCH_AREA']).median()['CAD_INCIDENT_ID']

x = pd.DataFrame({'DISPATCH_AREA': subset_median_daily.index, 'MAX': subset_max_daily, 'MEDIAN_2020': subset_median_daily})
y = pd.DataFrame({'DISPATCH_AREA': subset_median_daily_2015_2019.index, 'MEDIAN_2015_2019': subset_median_daily_2015_2019})
xy = x.merge(y, on='DISPATCH_AREA')
xy.to_csv('dispatch_values.csv', index=False)
xy

,DISPATCH_AREA,MAX,MEDIAN_2020,MEDIAN_2015_2019
0,B1,1,1.0,1.0
1,B2,2,1.0,1.0
2,B3,1,1.0,1.0
3,B5,2,1.0,1.0
4,CW,3,1.0,1.0
5,K1,1,1.0,1.0
6,K2,2,1.0,1.0
7,K3,1,1.0,1.0
8,K4,1,1.0,1.0
9,K5,1,1.0,1.0
